In [1]:
!mkdir data
!cp /kaggle/input/avinash-resume/Avinash_Resume.pdf ./data/Pdf1.pdf
!cp /kaggle/input/pdf-data/AU\ FS.pdf ./data/Pdf2.pdf
!cp /kaggle/input/pdf-data/Git-Cheat-Sheet.pdf ./data/Pdf3.pdf

# Mistral

In [2]:
!pip install -q pypdf transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index==0.9.39 langchain-community
!pip install -Uq bitsandbytes torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 97.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.7/325.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [4]:
documents = SimpleDirectoryReader('./data').load_data()
documents[0]

Document(id_='82ace0c8-2037-444d-95bd-6858814a6ff0', embedding=None, metadata={'page_label': '1', 'file_name': 'Pdf1.pdf', 'file_path': 'data/Pdf1.pdf', 'file_type': 'application/pdf', 'file_size': 125950, 'creation_date': '2024-12-28', 'last_modified_date': '2024-12-28', 'last_accessed_date': '2024-12-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Avinash R\n+91-9629149765 ravinash218@gmail.com LinkedIn GitHub Portfolio Hashnode\nEducation\nMepco Schlenk Engineering College\nBachelor of Engineering (B.Eng) in Computer Science And Engineering (CSE) CGPA: 8.79\nWork Experience\nAugrade Bangalore\nSoftware Engineer - Machine Learning Aug 2023 - Present\n• Dreamspace: Geometric Graph Based Building Information Modeling System| Python, Git, IfcOpenShe

In [5]:
prompt = """You are a Q&A assisstant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."""

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
import torch

llm = HuggingFaceLLM(
    context_window=512,#input size
    max_new_tokens=256,#output token
    generate_kwargs={'temperature':0.0, "do_sample":False},
    system_prompt=prompt,
    query_wrapper_prompt = query_wrapper_prompt,
    tokenizer_name='mistralai/Mistral-7B-Instruct-v0.1',
    model_name='mistralai/Mistral-7B-Instruct-v0.1',
    device_map='cuda',
    model_kwargs={'torch_dtype':torch.float16, "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 37.12 MiB is free. Process 4693 has 15.85 GiB memory in use. Of the allocated memory 15.45 GiB is allocated by PyTorch, and 107.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [ ]:
service_context = ServiceContext.from_defaults(chunk_size=512, llm=llm, embed_model=embed_model)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("Who is Avinash?")
print(response)